# Part II — Core Django  
## 13. Middleware, Sessions, Messages, and Cookies (How Django Manages State + Cross-Cutting Behavior)

This chapter explains the “invisible layer” that makes Django feel like a complete
web platform:

- **Middleware**: code that runs around every request/response (security, auth,
  sessions, CSRF, logging, localization, etc.).
- **Sessions**: how login state and per-user state survives across requests.
- **Cookies**: the browser-side mechanism that enables sessions (and many other
  behaviors).
- **Messages**: the “flash message” system (e.g., “Saved successfully!”) and how it
 ’s stored.

If you truly understand these four, you’ll be able to:
- debug auth/CSRF issues quickly,
- implement professional features like request IDs and timing headers,
- build good UX feedback loops,
- and configure secure production behavior with confidence.

---

## 13.0 Learning Outcomes

By the end of this chapter, you should be able to:

1. Explain how middleware wraps requests and why **order matters**.
2. Write modern (“new-style”) middleware with `__init__` and `__call__`.
3. Add cross-cutting behaviors:
   - request timing header
   - request ID correlation
   - security headers
4. Use sessions correctly:
   - `request.session[...]`
   - expiry, rotation, flush
   - understand session backends
5. Use cookies correctly:
   - `set_cookie`, `delete_cookie`
   - HttpOnly/Secure/SameSite implications
6. Use messages framework properly:
   - add messages in views
   - render in templates
   - understand storage backends
7. Write tests proving:
   - middleware headers exist
   - session values persist
   - messages appear after redirects

---

## 13.1 Middleware: The “Wrap Every Request” Pipeline

### 13.1.1 What middleware actually is (concrete mental model)

Middleware forms a chain like onion layers.

If you have middleware A then B then your view, the call order is:

```text
Request in:
  A before
    B before
      view
    B after
  A after
Response out
```

This is why middleware can:
- read/modify the request before it reaches your view
- read/modify the response after your view returns it
- short-circuit (return a response without calling the view)

### 13.1.2 Why order matters (real-world examples)

#### Example 1: Sessions + Authentication
- `SessionMiddleware` must run before `AuthenticationMiddleware`.
- Otherwise auth can’t load user identity from session, so `request.user` won’t be
  correct.

#### Example 2: CSRF depends on sessions/cookies + request context
- CSRF middleware relies on cookies/session and request processing. If it’s missing
  or misordered, POST requests can fail with 403.

#### Example 3: Compression / security headers should run late
- Middleware that modifies final response headers usually runs later so it sees the
  final response from view and other middleware.

---

## 13.2 Built-in Middleware: What Each One Does (The Ones You Should Recognize)

Your settings likely include:

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

Here’s what these do in plain language:

### 13.2.1 `SecurityMiddleware`
Adds important security behaviors and headers depending on settings, such as:
- redirects HTTP → HTTPS (if configured)
- HSTS header (Strict-Transport-Security) (if configured)
- content type sniffing protection (if configured)

It’s “policy enforcement” for security.

### 13.2.2 `SessionMiddleware`
Enables `request.session`.
- Reads session id from cookie
- Loads session data from backend
- Saves modified session at end of request

Without it:
- `request.session` won’t work
- most cookie-based login flows won’t work properly

### 13.2.3 `CommonMiddleware`
General web behaviors. Notable ones:
- `APPEND_SLASH`: if a URL is missing trailing slash and no match exists, Django can
  redirect to the slash version (e.g., `/about` → `/about/`)
- normalizes some URL patterns

This affects routing and redirects, so it matters for your URL design.

### 13.2.4 `CsrfViewMiddleware`
Protects against CSRF by requiring tokens on unsafe methods (POST/PUT/PATCH/DELETE)
for session-authenticated interactions.

### 13.2.5 `AuthenticationMiddleware`
Sets `request.user` based on session authentication.

### 13.2.6 `MessageMiddleware`
Enables `messages` framework (flash messages), storing messages between requests.

### 13.2.7 `XFrameOptionsMiddleware`
Protects against clickjacking by setting `X-Frame-Options` header (configurable).

---

## 13.3 Modern (“New-Style”) Middleware: Anatomy + Best Practices

Django’s modern middleware looks like this:

```python
class MyMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        # before view
        response = self.get_response(request)
        # after view
        return response
```

### 13.3.1 Why middleware is initialized once
- `__init__` runs at server startup.
- It’s a good place to set up static configuration.
- Do not do heavy work (DB calls, network calls) in `__init__`.

### 13.3.2 The “before/after” pattern is the key
- “before” code runs for every request before view
- “after” code runs for every request after view

### 13.3.3 Short-circuiting: returning early
Middleware can return a response without calling the view:

```python
from django.http import JsonResponse


class BlockRobotsMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        if request.headers.get("User-Agent", "").lower().find("bot") != -1:
            return JsonResponse({"error": "blocked"}, status=403)
        return self.get_response(request)
```

Use this carefully; it’s powerful but can be surprising if misused.

---

## 13.4 LAB 1 — Request ID Middleware (Professional Observability Baseline)

A request ID is a unique identifier assigned to each request. It lets you correlate:

- a user report (“that request failed”)
- server logs
- error monitoring events
- tracing systems

### 13.4.1 Create middleware that:
- reads incoming `X-Request-Id` if provided (from a load balancer)
- otherwise generates one
- stores it on `request.request_id`
- returns it in response header `X-Request-Id`

Create `config/middleware.py`:

```python
import uuid

from django.http import HttpRequest, HttpResponse


class RequestIdMiddleware:
    header_name = "X-Request-Id"

    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request: HttpRequest) -> HttpResponse:
        incoming = request.headers.get(self.header_name)
        request_id = incoming or uuid.uuid4().hex

        setattr(request, "request_id", request_id)

        response = self.get_response(request)
        response[self.header_name] = request_id
        return response
```

#### Explanation of important details
- `uuid.uuid4().hex` is a good request id:
  - reasonably unique
  - simple string
- Using incoming header supports upstream-generated IDs:
  - e.g., Nginx, Envoy, a cloud load balancer
- We attach it to `request` so views can use it if needed.

### 13.4.2 Add it to `MIDDLEWARE` (correct position)
Place it near the top so every later middleware and your view sees it.

In `config/settings.py`:

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "config.middleware.RequestIdMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

### 13.4.3 Verify with curl
```bash
curl -i http://127.0.0.1:8000/healthz/
```

You should see:

```text
X-Request-Id: <some hex>
```

Test incoming request id pass-through:

```bash
curl -i http://127.0.0.1:8000/healthz/ -H "X-Request-Id: demo-123"
```

Response should contain `X-Request-Id: demo-123`.

---

## 13.5 LAB 2 — Combine Request Timing + Request ID (Clean Headers Practice)

If you kept your `RequestTimingMiddleware`, keep both. If not, recreate it (slightly
refined) in `config/middleware.py` so all middleware live in one place.

Add:

```python
import time

from django.http import HttpRequest, HttpResponse


class RequestTimingMiddleware:
    header_name = "X-Request-Duration-Ms"

    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request: HttpRequest) -> HttpResponse:
        start = time.perf_counter()
        response = self.get_response(request)
        duration_ms = (time.perf_counter() - start) * 1000
        response[self.header_name] = f"{duration_ms:.2f}"
        return response
```

Register it below request id (so timing includes everything after it, which is fine):

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "config.middleware.RequestIdMiddleware",
    "config.middleware.RequestTimingMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

Verify:

```bash
curl -i http://127.0.0.1:8000/articles/html/
```

You should see both:
- `X-Request-Id`
- `X-Request-Duration-Ms`

---

## 13.6 Sessions: Persistent Per-User State (Server-Side by Default)

### 13.6.1 What a session is (mechanics)
- The browser stores a cookie: `sessionid=<random>`
- Django uses that to load session data from the session backend
- Your code reads/writes `request.session`

So you can store small bits of per-user state without exposing it to the user.

### 13.6.2 What sessions are used for
Common uses:
- login state (auth uses sessions)
- multi-step forms (“wizard”)
- shopping cart (small)
- user preferences (theme/language) (small)
- temporary flags (“just onboarded”) (short-lived)

**What not to store in sessions**
- huge data (it’s stored per request; can bloat DB/cache)
- sensitive secrets that must be rotated carefully (sessions can leak)
- permanent data better stored in DB model

### 13.6.3 Read/write session in a view
Add to `pages/views.py`:

```python
from django.http import JsonResponse
from django.views.decorators.http import require_POST


@require_POST
def set_theme(request):
    theme = request.POST.get("theme", "light")
    if theme not in {"light", "dark"}:
        return JsonResponse({"error": "invalid theme"}, status=400)

    request.session["theme"] = theme
    return JsonResponse({"status": "ok", "theme": theme})


def get_theme(request):
    return JsonResponse({"theme": request.session.get("theme", "light")})
```

Add to `pages/urls.py`:

```python
from . import views

urlpatterns = [
    # ...
    path("theme/", views.get_theme, name="get_theme"),
    path("theme/set/", views.set_theme, name="set_theme"),
]
```

Try it:

```bash
curl -i http://127.0.0.1:8000/theme/
```

Now set theme (this requires cookies to persist). Use `curl` cookie jar:

```bash
curl -i -c cookies.txt -b cookies.txt \
  -X POST http://127.0.0.1:8000/theme/set/ \
  -H "Content-Type: application/x-www-form-urlencoded" \
  --data "theme=dark"

curl -i -c cookies.txt -b cookies.txt http://127.0.0.1:8000/theme/
```

#### Explanation
- `-c cookies.txt` saves cookies from response.
- `-b cookies.txt` sends cookies in the next request.
- This simulates a browser maintaining session state.

### 13.6.4 When is a session saved?
Django saves the session when:
- you modify it (set a key)
- it’s not empty
- and the response is being returned

You can force a save by setting:

```python
request.session.modified = True
```

…but you typically don’t need to.

---

## 13.7 Session Expiry, Rotation, and “Logging Out”

### 13.7.1 Session expiry
Sessions can expire based on:
- browser session lifetime
- configured age (`SESSION_COOKIE_AGE`)
- explicit expiry (`request.session.set_expiry(...)`)

Example: expire after 10 minutes:

```python
request.session.set_expiry(600)
```

### 13.7.2 Clearing session data
- Remove a key:

```python
del request.session["theme"]
```

- Clear all keys but keep session:

```python
request.session.clear()
```

- Flush session (clears data and rotates session key, effectively new session):

```python
request.session.flush()
```

`flush()` is often used on logout for safety.

### 13.7.3 Session fixation (security concept you should know)
Session fixation is when an attacker forces a victim to use a session ID chosen by
the attacker. Good auth systems rotate session IDs after login.

Django rotates session on login by default in typical flows, which helps.

---

## 13.8 Session Backends (Where Session Data Is Stored)

Django can store sessions in different places:

### 13.8.1 Database-backed sessions (default in many setups)
- Stored in `django_session` table.
- Easy and reliable.
- Can become large with heavy traffic unless cleaned.

### 13.8.2 Cached sessions (Redis/memcache)
- Faster.
- Great at scale.
- Requires cache infrastructure and persistence strategy.

### 13.8.3 File-based sessions
- Stored on disk.
- Not common in production.

### 13.8.4 Cookie-based sessions (signed cookies)
- Stored client-side but cryptographically signed.
- No server storage.
- Limited size and sensitive to key rotation.
- Data is visible to user (even if tamper-proof), so do not store secrets.

**Industry preference**
- DB sessions for simple deployments
- Redis-backed sessions for scale
- cookie sessions only for very specific use cases

(We’ll implement production session choices later in deployment chapters.)

---

## 13.9 Cookies: How to Use Them Safely

Cookies are how browsers persist small state and how sessions work.

### 13.9.1 Set a cookie
In a view:

```python
from django.http import JsonResponse


def set_cookie_demo(request):
    response = JsonResponse({"status": "ok"})
    response.set_cookie(
        "promo_seen",
        "1",
        max_age=30 * 24 * 60 * 60,
        httponly=True,
        samesite="Lax",
    )
    return response
```

### 13.9.2 Delete a cookie
```python
response.delete_cookie("promo_seen")
```

### 13.9.3 Cookie security attributes (must understand)

- `HttpOnly=True`:
  - JS cannot read it
  - reduces impact of XSS stealing cookies
- `Secure=True`:
  - cookie only sent over HTTPS
  - should be True in production
- `SameSite`:
  - `Lax` blocks most cross-site “form POST” style CSRF while still allowing normal
    navigation
  - `Strict` is more restrictive (can break legitimate flows)
  - `None` allows cross-site; must be Secure; increases CSRF risk if not designed

**Important reality:** cookies + session auth require a CSRF strategy. Django’s CSRF
middleware is designed for this.

---

## 13.10 Signed Values (When You Must Store Something Client-Side)

Sometimes you want to store a small client-side value that must not be tampered
with (not a session).

Django provides signing utilities.

Example (demo only):

```python
from django.core import signing
from django.http import JsonResponse


def signed_cookie_set(request):
    value = {"tier": "free", "experiment": "A"}
    signed = signing.dumps(value)
    response = JsonResponse({"status": "ok"})
    response.set_cookie("exp", signed, samesite="Lax")
    return response


def signed_cookie_get(request):
    signed = request.COOKIES.get("exp")
    if not signed:
        return JsonResponse({"exp": None})

    try:
        value = signing.loads(signed, max_age=7 * 24 * 60 * 60)
    except signing.BadSignature:
        return JsonResponse({"error": "tampered"}, status=400)
    except signing.SignatureExpired:
        return JsonResponse({"error": "expired"}, status=400)

    return JsonResponse({"exp": value})
```

**Key security note:** signing prevents tampering, not reading. Users can still see
the data. Do not store secrets there.

---

## 13.11 Messages Framework: Flash Messages Between Requests

Messages are for “one-time feedback,” like:
- “Article created successfully.”
- “Password reset email sent.”
- “You don’t have permission.”

### 13.11.1 Why messages exist
After PRG (POST → redirect → GET), your POST view ends with a redirect.
You can’t easily show “Saved!” on the target page unless you carry it across the
redirect.

Messages solve exactly that:
- POST sets a message
- redirect happens
- GET renders template and displays message
- message disappears after being shown

### 13.11.2 How to add messages
In a view:

```python
from django.contrib import messages

messages.success(request, "Saved successfully.")
messages.error(request, "Something went wrong.")
messages.info(request, "FYI: ...")
messages.warning(request, "Be careful.")
```

### 13.11.3 Message levels and tags
Django associates levels with tags, e.g. `success`, `error`. In templates, you used:

```django
<li class="message {{ message.tags }}">{{ message }}</li>
```

This enables CSS per message type.

### 13.11.4 Where messages are stored
Django uses a message storage backend. The default is typically **FallbackStorage**:
- tries cookie storage first
- falls back to session storage if needed

Practical consequences:
- messages must be small (cookie size limits)
- sessions must work reliably for larger messages

---

## 13.12 LAB 3 — “Theme Preference” + Messages + Sessions (End-to-End UX)

We’ll build a real UX flow:

- A page lets a logged-in user choose a theme (light/dark).
- Choice is stored in session.
- User gets a “Theme updated” flash message after redirect.
- Base template adds a CSS class based on theme.

### 13.12.1 Add a view with PRG + messages
In `pages/views.py`:

```python
from django.contrib import messages
from django.shortcuts import redirect, render
from django.views.decorators.http import require_http_methods


@require_http_methods(["GET", "POST"])
def theme_settings(request):
    if request.method == "POST":
        theme = request.POST.get("theme", "light")
        if theme not in {"light", "dark"}:
            messages.error(request, "Invalid theme selected.")
            return redirect("pages:theme_settings")

        request.session["theme"] = theme
        messages.success(request, f"Theme updated to {theme}.")
        return redirect("pages:theme_settings")

    current = request.session.get("theme", "light")
    return render(
        request,
        "pages/theme_settings.html",
        {"current_theme": current},
    )
```

Add URL in `pages/urls.py`:

```python
path("theme-settings/", views.theme_settings, name="theme_settings"),
```

### 13.12.2 Add template
Create `pages/templates/pages/theme_settings.html`:

```django
{% extends "base.html" %}

{% block title %}Theme Settings{% endblock %}

{% block content %}
  <h1>Theme Settings</h1>

  <p>Current theme: <code>{{ current_theme }}</code></p>

  <form method="post">
    {% csrf_token %}

    <label>
      <input
        type="radio"
        name="theme"
        value="light"
        {% if current_theme == "light" %}checked{% endif %}
      />
      Light
    </label>

    <label>
      <input
        type="radio"
        name="theme"
        value="dark"
        {% if current_theme == "dark" %}checked{% endif %}
      />
      Dark
    </label>

    <button type="submit">Save</button>
  </form>
{% endblock %}
```

### 13.12.3 Apply theme to the page (base template)
Update `templates/base.html` `<body>` tag to use session theme:

```django
<body class="theme-{{ request.session.theme|default:'light' }}">
```

Important: `request.session.theme` works because templates can access dict keys as
attributes in many cases. If you want to be explicit and safe, use:

```django
<body class="theme-{{ request.session.theme|default:'light' }}">
```

Or pass a `theme` variable via context processor (more professional as it centralizes
access). We’ll do that next.

### 13.12.4 (Better) Add a context processor for theme
Create `config/context_processors.py` (if you already have `site_info`, add this):

```python
def theme(request):
    return {"theme": request.session.get("theme", "light")}
```

Register in settings (TEMPLATES context processors list):

```python
"config.context_processors.theme",
```

Now in `base.html`:

```django
<body class="theme-{{ theme }}">
```

This is cleaner and avoids repeating session access patterns.

### 13.12.5 Add a tiny CSS difference
In `static/css/site.css`:

```css
.theme-dark .container {
  background: #111;
  color: #eee;
}

.theme-dark .nav {
  background: #222;
}
```

Now your session preference affects UI.

---

## 13.13 Testing Middleware + Sessions + Messages (Professional Safety Net)

### 13.13.1 Test request ID header exists
Create `pages/tests_middleware.py`:

```python
from django.test import TestCase


class MiddlewareHeaderTests(TestCase):
    def test_request_id_header_present(self):
        response = self.client.get("/healthz/")
        self.assertEqual(response.status_code, 200)
        self.assertIn("X-Request-Id", response.headers)

    def test_request_id_passthrough(self):
        response = self.client.get(
            "/healthz/",
            headers={"X-Request-Id": "test-req-123"},
        )
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.headers["X-Request-Id"], "test-req-123")
```

### 13.13.2 Test session persists across requests
Add to `pages/tests_sessions.py`:

```python
from django.test import TestCase
from django.urls import reverse


class SessionTests(TestCase):
    def test_theme_stored_in_session(self):
        set_url = reverse("pages:theme_settings")
        get_url = reverse("pages:theme_settings")

        response = self.client.post(set_url, {"theme": "dark"}, follow=True)
        self.assertEqual(response.status_code, 200)

        response2 = self.client.get(get_url)
        self.assertContains(response2, "Current theme:")
        self.assertContains(response2, "dark")
```

### 13.13.3 Test messages show up after redirect
Messages are easiest to test by checking response content after `follow=True`.

```python
from django.test import TestCase
from django.urls import reverse


class MessageTests(TestCase):
    def test_theme_update_message(self):
        url = reverse("pages:theme_settings")
        response = self.client.post(url, {"theme": "dark"}, follow=True)
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Theme updated to dark.")
```

---

## 13.14 Production-Security Settings You Should Know (Now)

Even though we’ll do a full security chapter later, you must know where session and
cookie security is controlled.

In `config/settings.py`, common production settings include:

```python
SESSION_COOKIE_HTTPONLY = True
SESSION_COOKIE_SECURE = True
CSRF_COOKIE_SECURE = True
CSRF_COOKIE_HTTPONLY = False
SESSION_COOKIE_SAMESITE = "Lax"
CSRF_COOKIE_SAMESITE = "Lax"
```

### Explanations:
- `SESSION_COOKIE_HTTPONLY=True`: JS can’t read session cookie.
- `SESSION_COOKIE_SECURE=True`: send session cookie only over HTTPS.
- `CSRF_COOKIE_HTTPONLY` is typically False because JS may need to read CSRF token
  in some architectures; Django’s default patterns often don’t require it.
- SameSite Lax is a good default for many classic Django apps.

**Important:** Don’t flip these randomly in development if you’re using HTTP, or
cookies won’t be sent and login/session will appear broken.

---

## 13.15 Common Problems and How to Debug Them Fast

### Problem A: “I’m logged in but keep getting logged out”
Common causes:
- cookies not being stored (browser settings, domain mismatch)
- `SESSION_COOKIE_SECURE=True` while using HTTP
- `ALLOWED_HOSTS` or proxy misconfig causing redirects to another host
- multiple servers without shared session backend (in production)

Debug steps:
1. Inspect response cookies in browser devtools.
2. Check if session cookie exists and is sent on requests.
3. Check `SESSION_COOKIE_SECURE` vs HTTPS usage.

### Problem B: CSRF token missing/invalid (403 on POST)
Common causes:
- forgot `{% csrf_token %}`
- submitting POST cross-site
- messing with cookies or using wrong domain
- caching pages with tokens incorrectly

Debug steps:
- view page source: does form contain csrf hidden input?
- ensure `CsrfViewMiddleware` is enabled
- verify cookies include `csrftoken` (in many setups)

### Problem C: Messages not showing
Common causes:
- messages partial not included
- MessageMiddleware missing
- template context processor missing
- using redirect but not rendering message on next page

Debug:
- ensure base template includes messages partial
- ensure settings include middleware + context processor

---

## 13.16 Exercises (Do These Before Proceeding)

1. Add a middleware that adds a `X-App-Name` header:
   - header value from settings (e.g., `APP_NAME = "MySite"`)
   - test that it appears on responses

2. Add a “dismissed banner” cookie:
   - endpoint sets cookie `banner_dismissed=1`
   - base template hides banner if cookie exists
   - explain why cookie is suitable here (client-only UI preference)

3. Add session expiry behavior:
   - set theme session to expire in 1 hour when changed
   - explain why you might want shorter expiry for some data

4. Write one test that confirms:
   - session cookie is set after a view that writes session

---

## 13.17 Chapter Summary

- Middleware is an ordered request/response wrapper chain; order is not optional.
- Sessions give server-side per-user state; cookies carry the session id.
- Cookie attributes (HttpOnly/Secure/SameSite) are security-critical.
- Messages are flash notifications stored via a backend (often cookie/session
  fallback) and designed for PRG flows.
- Professional Django apps almost always implement:
  - request IDs
  - timing headers
  - secure session cookie settings in production
  - messages for UX feedback

---

Next chapter: **Part II — Chapter 14: Static Files and Media Files**  
We’ll formalize static file handling (collectstatic, best structure), introduce media
uploads with `MEDIA_ROOT/MEDIA_URL`, secure upload validation, and the correct dev vs
production serving strategies.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='12. authentication_and_authorization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='14. static_files_and_media_files.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
